In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.0.0 cuda: True


In [2]:
import matplotlib.pyplot as plt
## Load dataset
data_path = "../gmm_dataset_3c"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 1
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 100
BATCH_SIZE = 1
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:1')

PATH = 'ag-%dSample' % 10
Test_Params = (NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc import *
from ag_ep import ELBO

Reparameterized = False

# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)

models = (oneshot_eta, enc_eta, enc_z)

In [4]:
enc_z.load_state_dict(torch.load("../weights/enc-z-%s" % PATH))
enc_eta.load_state_dict(torch.load("../weights/enc-eta-%s" % PATH))
oneshot_eta.load_state_dict(torch.load("../weights/oneshot-eta-%s" % PATH))

In [5]:
from testing import *

# DBs_eta, DBs_z, ELBOs, Ratios, LLs = Test(models, ELBO, Data, Model_Params, Test_Params)

In [ ]:
from local_gibbs import *
from global_gibbs import *
from gibbs import *
gibbs_z = Gibbs_z(K, CUDA, DEVICE)
gibbs_eta = Gibbs_eta(K, D, CUDA, DEVICE)

models = (gibbs_eta, gibbs_z)

In [ ]:

DBs_eta, DBs_z, ELBOs, Ratios, LLs = Test(models, Gibbs, Data, Model_Params, Test_Params)

In [ ]:
fs = 10
fig = plt.figure(figsize=(fs * 2, fs))
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(ELBOs.cpu().data.numpy(), 'o-', label='elbo')
ax1.legend()
ax1.set_xlabel('Gibbs Update')
ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(DBs_eta.cpu().data.numpy(), 'o-', label='DB_eta')
ax2.plot(DBs_z.cpu().data.numpy(), 'o-', label='DB_z')
ax2.set_xlabel('Gibbs Update')
ax2.legend()
plt.savefig('gibbs_results.pdf')

In [ ]:
Ratios

In [ ]:
L = Ratios.shape[0]
EE = np.zeros(L)
for i in range(L):
    EE[i] = Ratios[:i+1].sum().cpu().data.numpy()

In [ ]:
plt.plot(EE)